In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import modules.parsers as parsers
import glob

In [2]:
import configparser
import os
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.3")\
                     .master("spark://pop-os.localdomain:7077")\
                     .enableHiveSupport()\
                     .getOrCreate()
sc = spark.sparkContext

22/04/12 00:18:04 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.2.67 instead (on interface wlp59s0)
22/04/12 00:18:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/paul/.ivy2/cache
The jars for the packages stored in: /home/paul/.ivy2/jars
:: loading settings :: url = jar:file:/home/paul/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d820c67-1cf4-457c-a8d6-a8690af6dc29;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;2.7.3 in central
	found org.apache.hadoop#hadoop-common;2.7.3 in central
	found org.apache.hadoop#hadoop-annotations;2.7.3 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found

In [4]:
# File is 1.5 Gb will take a couple minutes to load into spark
reddit_data_url = "s3a://paul-udacity-capstone/reddit/001.ndjson" # For Testing
# reddit_data_url = "s3a://paul-udacity-capstone/reddit/RC_2018_01_01"
df_reddit = spark.read.json(reddit_data_url)

In [5]:
df_reddit.count()

22/04/12 00:18:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


1000

In [6]:
git_data_url = "s3a://paul-udacity-capstone/git-dump/freeCodeCamp/freeCodeCamp.json" # For Testing
# git_data_url = "s3a://paul-udacity-capstone/git-dump/*/*.json"
df_git = spark.read.json(git_data_url)

In [7]:
df_git.count()

4789

## Working with git repos

In [8]:
git_repo_df = parsers.df_parse_email(df_git,  "author_email")

In [9]:
git_repo_df.limit(5).toPandas()

,author_email,author_name,commits,remote_url,email_username,email_domain
0,0nitinchauhan@gmail.com,Nitin Chauhan,3,https://github.com/freeCodeCamp/freeCodeCamp.git,0nitinchauhan,gmail.com
1,0x0936@users.noreply.github.com,Robert Richey,1,https://github.com/freeCodeCamp/freeCodeCamp.git,0x0936,users.noreply.github.com
2,10049458+dallyingllama@users.noreply.github.com,dallyingllama,2,https://github.com/freeCodeCamp/freeCodeCamp.git,10049458+dallyingllama,users.noreply.github.com
3,1010636+JeremyBlanc@users.noreply.github.com,Jeremias Blanco-Choncén,2,https://github.com/freeCodeCamp/freeCodeCamp.git,1010636+JeremyBlanc,users.noreply.github.com
4,101513652+anjiqbal@users.noreply.github.com,Anjum Iqbal,1,https://github.com/freeCodeCamp/freeCodeCamp.git,101513652+anjiqbal,users.noreply.github.com


## Working with reddit data

In [10]:
reddit_df = parsers.df_extract_url(df_reddit, "body")
reddit_df = parsers.df_parse_domainname(reddit_df, "url")

In [11]:
reddit_df.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_created_utc: long (nullable = true)
 |-- author_flair_background_color: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_richtext: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- a: string (nullable = true)
 |    |    |-- e: string (nullable = true)
 |    |    |-- t: string (nullable = true)
 |    |    |-- u: string (nullable = true)
 |-- author_flair_template_id: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_flair_text_color: string (nullable = true)
 |-- author_flair_type: string (nullable = true)
 |-- author_fullname: string (nullable = true)
 |-- author_patreon_flair: boolean (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- can_mod_post: boolean (nullable = true)
 |-

In [12]:
reddit_df = reddit_df.withColumn("url",reddit_df.url.cast('string'))

In [13]:
# reddit_df3 = reddit_df[["score", "body", "url", "domain_name"]].filter( F.col("body") != "" )

In [14]:
#reddit_df3.show()

In [15]:
# 

In [16]:
# reddit_df[["url"]].filter( F.col("url") != "" ).count().show()

In [17]:
len(reddit_df.head(343))

343

In [24]:
import pprint
reddit_sec =  reddit_df.head(343)[-1]
for i in reddit_sec:
    print(i)

None
Azurealy
None
1443479354

jax
[]
None

dark
text
t2_qst8f
False
The purpose was to reduce people feeling unmotivated from being autofilled. So this was to have people say "well i dont want my support rank to decrease " but instead people just dont care and getting secondary or autofill makes people not care about that game because it isnt the main role and wont lose much. All in all its the biggest failure ive ever seen come from riot
True
False
False
None
0
1550188805
None
false
0
Row(gid_1=0, gid_2=0, gid_3=0)
eghu5qh
False
t3_aqlth5
True
t3_aqlth5
/r/leagueoflegends/comments/aqlth5/position_ranks_failed_to_motivate_players/eghu5qh/
False
None
1554229777
1
True
False
leagueoflegends
t5_2rfxx
r/leagueoflegends
public




In [34]:
problem_json2 = """
[{
  "author": "Argonnerwald",
  "author_created_utc": 1502970738,
  "author_flair_background_color": null,
  "author_flair_css_class": null,
  "author_flair_richtext": [],
  "author_flair_template_id": null,
  "author_flair_text": null,
  "author_flair_text_color": null,
  "author_flair_type": "text",
  "author_fullname": "t2_atyby9r",
  "author_patreon_flair": false,
  "body": "Except this ranking is already based heavily on economic factors. It doesn't take into account things like loneliness, a sense of purpose and meaning, integration into a community etc. Iceland -#4 on this ranking - has second highest antidepression rate in world. ",
  "can_gild": true,
  "can_mod_post": false,
  "collapsed": false,
  "collapsed_reason": null,
  "controversiality": 0,
  "created_utc": 1550188805,
  "distinguished": null,
  "edited": false,
  "gilded": 0,
  "gildings": {
    "gid_1": 0,
    "gid_2": 0,
    "gid_3": 0
  },
  "id": "eghu5qk",
  "is_submitter": false,
  "link_id": "t3_aqpiyl",
  "no_follow": false,
  "parent_id": "t1_eghn7v4",
  "permalink": "\/r\/europe\/comments\/aqpiyl\/european_happiness\/eghu5qk\/",
  "quarantined": false,
  "removal_reason": null,
  "retrieved_on": 1554229777,
  "score": 4,
  "send_replies": true,
  "stickied": false,
  "subreddit": "europe",
  "subreddit_id": "t5_2qh4j",
  "subreddit_name_prefixed": "r\/europe",
  "subreddit_type": "public"
}]
"""

In [30]:
problem_json = """
[{
  "author": "psswrd12345",
  "author_created_utc": 1465257237,
  "author_flair_background_color": "",
  "author_flair_css_class": "",
  "author_flair_richtext": [],
  "author_flair_template_id": null,
  "author_flair_text": " ",
  "author_flair_text_color": "dark",
  "author_flair_type": "text",
  "author_fullname": "t2_yilo2",
  "author_patreon_flair": false,
  "body": "Semi serious question - do you think CSW is 1) insane, 2) has been on a multiyear cocaine binge, or 3) both.  His twitter barf on JP in particular is almost mesmerizing in its lunacy.\n\n[https:\/\/twitter.com\/ProfFaustus\/status\/1096018468858667008](https:\/\/twitter.com\/ProfFaustus\/status\/1096018468858667008)",
  "can_gild": true,
  "can_mod_post": false,
  "collapsed": false,
  "collapsed_reason": null,
  "controversiality": 0,
  "created_utc": 1550188805,
  "distinguished": null,
  "edited": 1550189033,
  "gilded": 0,
  "gildings": {
    "gid_1": 0,
    "gid_2": 0,
    "gid_3": 0
  },
  "id": "eghu5qi",
  "is_submitter": false,
  "link_id": "t3_aqgo09",
  "no_follow": true,
  "parent_id": "t3_aqgo09",
  "permalink": "\/r\/ethtrader\/comments\/aqgo09\/daily_general_discussion_february_14_2019\/eghu5qi\/",
  "quarantined": false,
  "removal_reason": null,
  "retrieved_on": 1554229777,
  "score": 9,
  "send_replies": true,
  "stickied": false,
  "subreddit": "ethtrader",
  "subreddit_id": "t5_37jgj",
  "subreddit_name_prefixed": "r\/ethtrader",
  "subreddit_type": "public"
}]
"""

In [38]:
# print(problem_json2)

In [41]:
import json
my_problem = json.loads(problem_json)
print(my_problem)

JSONDecodeError: Invalid control character at: line 14 column 195 (char 561)

In [19]:
reddit_df.limit(343).filter( F.col("body") != "" ).show() # 344 is working

+--------+--------------------+--------------+------------------+-----------------------------+----------------------+---------------------+------------------------+--------------------+-----------------------+-----------------+---------------+--------------------+--------------------+--------+------------+---------+----------------+----------------+-----------+-------------+----------+------+---------+-------+------------+---------+---------+----------+--------------------+-----------+--------------+------------+-----+------------+--------+--------------------+------------+-----------------------+--------------+--------------------+--------------+
|archived|              author|author_cakeday|author_created_utc|author_flair_background_color|author_flair_css_class|author_flair_richtext|author_flair_template_id|   author_flair_text|author_flair_text_color|author_flair_type|author_fullname|author_patreon_flair|                body|can_gild|can_mod_post|collapsed|collapsed_reason|controve

In [20]:
reddit_df[0]

Column<b'archived'>

In [21]:
reddit_df[reddit_df['url']==""].show()

+--------+--------------------+--------------+------------------+-----------------------------+----------------------+---------------------+------------------------+--------------------+-----------------------+-----------------+---------------+--------------------+--------------------+--------+------------+---------+----------------+----------------+-----------+-------------+----------+------+---------+-------+------------+---------+---------+----------+--------------------+-----------+--------------+------------+-----+------------+--------+--------------------+------------+-----------------------+--------------+---+-----------+
|archived|              author|author_cakeday|author_created_utc|author_flair_background_color|author_flair_css_class|author_flair_richtext|author_flair_template_id|   author_flair_text|author_flair_text_color|author_flair_type|author_fullname|author_patreon_flair|                body|can_gild|can_mod_post|collapsed|collapsed_reason|controversiality|created_utc

In [22]:
reddit_df.groupBy("score").count().show()

+-----+-----+
|score|count|
+-----+-----+
|   26|    1|
|   29|    1|
|   19|    5|
|    0|   34|
|  113|    1|
|   22|    4|
|    7|   14|
|   -7|    1|
|   32|    2|
|   31|    1|
|   39|    1|
|   25|    1|
|  124|    1|
| 2297|    1|
|   71|    1|
|    6|   21|
|    9|    7|
|   63|    1|
|   17|    4|
|   41|    1|
+-----+-----+
only showing top 20 rows



In [23]:
reddit_df2.show()

NameError: name 'reddit_df2' is not defined

In [ ]:
reddit_df.limit(5).toPandas()

## Data Integration Time
Separately grouBy the domain names, one for reddit, one for git then perform join on the dataframes 

In [ ]:
# reddit_domains_grouped = reddit_df.filter( F.col("url") != "" ).groupBy("domain_name").count()
# reddit_domains_grouped = reddit_domains_grouped.withColumnRenamed("count", "reddit_domain_count")

In [ ]:
reddit_df.printSchema()

In [ ]:
# reddit_domains_grouped = reddit_df.groupBy("domain_name").count()
reddit_df[["id"]].show()

In [ ]:
# reddit_df.createOrReplaceTempView("reddit_data")
# spark.sql( 'SELECT * FROM reddit_data where url == ""').show()

In [ ]:
#reddit_df.createOrReplaceTempView("reddit_data")
# spark.sql( 'SELECT * FROM reddit_data where url != ""').show()

In [ ]:
reddit_df.show()

In [ ]:
reddit_df.count()

In [ ]:
# def blank_as_null(x):
#     return F.when(F.col(x) != "", F.col(x)).otherwise(None)
# reddit_df = reddit_df.withColumn("url", blank_as_null("url"))

In [ ]:
# reddit_df = reddit_df.filter( F.col("url").isNotNull() )

In [ ]:
reddit_df.count()

In [5]:
# reddit_df.filter(F.col("url") == "").show()
# reddit_df.createOrReplaceTempView("reddit_data")
# spark.sql( 'SELECT * FROM reddit_data where url == ""').show()
reddit_df = reddit_df[["body", "url"]].filter( reddit_df["url"] == "")

NameError: name 'reddit_df' is not defined

In [ ]:
# reddit_df.count()

In [ ]:
# def blank_as_null(x):
#     return F.when(F.col(x) == "", F.col(x)).otherwise(None)
# reddit_df = reddit_df.withColumn("url", blank_as_null("url"))

In [ ]:
reddit_df.filter( F.col("url") == None).count()

In [ ]:
# reddit_df.filter( F.col("url") == "").count()

In [ ]:
# reddit_domains_grouped = reddit_df.filter( F.col("url") != "" )

In [ ]:
# reddit_domains_grouped.show()

In [ ]:
git_domains_grouped = git_repo_df.groupBy("email_domain").count()
git_domains_grouped = git_domains_grouped.withColumnRenamed("count", "git_domain_count")

In [ ]:
# reddit_domains_grouped.show(3)

In [ ]:
git_domains_grouped.show(10, False)

In [ ]:
domains_list = reddit_domains_grouped[["domain_name", "reddit_domain_count"]].\
    union(git_domains_grouped[["email_domain", "git_domain_count"]])

In [ ]:
joined_domains = reddit_domains_grouped.alias("reddit")\
  .join(git_domains_grouped.alias("git"), F.col("git.email_domain") == F.col("reddit.domain_name"))

In [ ]:
git_domains_grouped.show()

In [ ]:
joined_domains.show()

In [ ]:
unioned_domains = reddit_domains_grouped.unionByName(git_domains_grouped.withColumnRenamed("email_domain", "domain_name"),  allowMissingColumns=True)

In [ ]:
unioned_domains.show()

In [ ]:
unioned_domains = reddit_domains_grouped.unionByName(git_domains_grouped)